### Загружаем модель для детекции уклончивости с huggingface

[link for english model](https://huggingface.co/alenaa/evasiveness)

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("alenaa/evasiveness")
model = AutoModelForSequenceClassification.from_pretrained("alenaa/evasiveness")

/Users/alena.komarova/opt/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
tokenizer.save_pretrained("local-pt-checkpoint", safe_serialization=False)  
model.save_pretrained("local-pt-checkpoint", safe_serialization=False)

### Конвертируем в формат onnx

In [ ]:
pip install 'transformers[onnx]'

In [ ]:
!conda install onnxruntime -c conda-forge -y

In [ ]:
import subprocess  

subprocess.run(f"python -m transformers.onnx --model=local-pt-checkpoint --feature=sequence-classification onnx/".split())

Local PyTorch model found.
Framework not requested. Using torch to export to ONNX.


CompletedProcess(args=['python', '-m', 'transformers.onnx', '--model=local-pt-checkpoint', '--feature=sequence-classification', 'onnx/'], returncode=-11)

In [4]:
import onnxruntime 

onnx_session = onnxruntime.InferenceSession(
    'onnx/model.onnx',
    providers=['CPUExecutionProvider'])

### Тестируем модели

Предсказываем с помощью ONNX модели

In [5]:
import numpy as np
import torch 
import time 
import tracemalloc

def predict_onnx(feed):
    start_time = time.time()
    tracemalloc.start()
    output = onnx_session.run(None, feed)
    mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end_time = time.time() - start_time
    output_tensor = torch.tensor(output[0], dtype=torch.float32)
    softmax_output = torch.nn.functional.softmax(output_tensor, dim=-1)
    predictions = np.squeeze(softmax_output.numpy())
    return(np.argmax(predictions, axis=1)), end_time, mem


Предсказываем с помощью обычной модели

In [12]:
def predict(inputs):
    model.eval()
    with torch.no_grad():
        tracemalloc.start()
        start_time = time.time()
        outputs = model(**inputs)
        end_time = time.time() - start_time
        mem = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        logits = outputs.logits
        
        predictions = torch.argmax(logits, dim=-1)
        
        return predictions, end_time, mem

Проверим точность и время на сгенерированном датасете

In [7]:
import pandas as pd

data = pd.read_csv('english_data_200.csv', index_col = 0)
Q = data.Question.values.tolist()
A = data.Answer.values.tolist()

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("local-pt-checkpoint")
model = AutoModelForSequenceClassification.from_pretrained("local-pt-checkpoint")

In [9]:
inputs = tokenizer(Q, A, truncation=True, padding=True, max_length=512, return_tensors="pt")
feed = dict(
        input_ids=np.array(inputs["input_ids"]).astype("int64"),
        attention_mask=np.array(inputs["attention_mask"]).astype("int64"))

In [10]:
from sklearn.metrics import accuracy_score

test = data.Label.values.tolist()

def print_results(prediction_func, inp):
    test = data.Label.values.tolist()
    preds, time_, mem = prediction_func(inp)
    mse = accuracy_score(test, preds.tolist())
    if prediction_func == predict:
        model_type = 'Simple Model'
    else:
        model_type = 'ONNX Model'
    print(f'{model_type}')
    print('MSE: ', mse)
    print('Time: ', time_)
    print('Memory: ', mem)

In [13]:
print_results(predict, inputs)

Simple Model
MSE:  0.7548076923076923
Time:  3.149432897567749
Memory:  (5114, 9582)


In [14]:
print_results(predict_onnx, feed)

ONNX Model
MSE:  0.7548076923076923
Time:  2.1993188858032227
Memory:  (3129, 5582)


Попробуем оптимизировать модель onnx

In [ ]:
!pip install onnxruntime_tools

In [16]:
from onnxruntime_tools import optimizer

optimized_model = optimizer.optimize_model("onnx/model.onnx", model_type='bert')
optimized_model.save_model_to_file("onnx/opt_model.onnx")
onnx_session = onnxruntime.InferenceSession(
    'onnx/opt_model.onnx',
    providers=['CPUExecutionProvider'])
print_results(predict_onnx, feed)


ONNX Model
MSE:  0.7548076923076923
Time:  1.9741268157958984
Memory:  (1992, 2143)
